In [41]:
import pandas as pd
import pgeocode
import numpy as np

In [42]:
def get_distances(df, address_type):
    distances = []
    for row in df.itertuples():
        if address_type=='PPMA':
            mf = row.ADDR_1_masterfile_PPMA
        else:
            mf = row.ADDR_1_masterfile_POLO
        ins = row.ADDR_1_insurance
        try:
            distance = get_distance(mf, ins)
            distances.append(distance)
        except:
            pass
    avg = sum(distances)/len(distances)
    print(f'Average distance between addressses is {avg} miles.')
    return(distances)

In [43]:
DF = pd.read_csv('polo_ppma_no_matches.csv')

In [44]:
DF.dtypes

Unnamed: 0                  int64
ADDR_1_masterfile_POLO     object
ADDR_2_masterfile_POLO     object
ADDR_3_masterfile_POLO     object
CITY_masterfile_POLO       object
DATE_masterfile_POLO       object
ME                          int64
STATE_masterfile_POLO      object
ZIP_masterfile_POLO         int64
ADDR_1_insurance           object
ADDR_1_masterfile_PPMA     object
ADDR_2_insurance_PPMA      object
ADDR_2_masterfile_PPMA     object
ADDR_3_insurance           object
ADDR_3_masterfile_PPMA     object
CITY_insurance             object
CITY_masterfile_PPMA       object
DATE_insurance             object
DATE_masterfile_PPMA       object
STATE_insurance_PPMA       object
STATE_masterfile_PPMA      object
ZIP_insurance              object
ZIP_masterfile_PPMA       float64
dtype: object

In [46]:
def get_distances(df, ziptype):
    distances = []
    TOTAL = df.shape[0]
    count =0
    DIST=pgeocode.GeoDistance('US')
    dict_list = []
    num_list =[]
    for row in DF.itertuples():
        new_dict={}
        new_dict["ME"]=row.ME
        count += 1
        if count % 100 == 0:
            print(count)
        if int(count/TOTAL*100) in [10,20,30,40,50,60,70,80,90] and int(count/TOTAL*100) not in num_list:
            num_list.append(int(count/TOTAL*100))
            print (f'{int(count/TOTAL*100)}% done!')
        try:
            if ziptype == "ppma":
                mf = int(row.ZIP_masterfile_PPMA)
            else:
                mf = int(row.ZIP_masterfile_POLO)
            ins = int(row.ZIP_insurance)
        except:
            mf='nan'
            ins='nan'
    #     print(f'{mf} {ins}')
        try:
            ZIP_distance = DIST.query_postal_code(mf, ins)*0.621371
    #         print(ZIP_distance)
            if np.isnan(ZIP_distance)==False:
                distances.append(ZIP_distance)
        except:
            ZIP_distance = 'nan'
        new_dict['ZIP_distance_'+ ziptype]=ZIP_distance
        dict_list.append(new_dict)
    avg = sum(distances)/len(distances)
    print(f'Average distance between {ziptype} addressses is {avg} miles.')
    return(dict_list, distances)

In [47]:
polo_dict_list, polo_distances = get_distances(DF, 'polo')

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
10% done!
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
20% done!
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
30% done!
17500
17600
17700
17800
17900
1800

In [48]:
polo_dict = pd.DataFrame(polo_dict_list)
# ppma_dist = pd.DataFrame(dict_list2)

In [51]:
updated_distances = pd.merge(zipcode_distances, polo_dict, on='ME', suffixes = ("_old", "_new"))

In [15]:
dists = pd.merge(polo_dist, ppma_dist, on ="ME", suffixes = ('_polo', '_ppma'))

In [17]:
total = pd.merge(dists, DF, on="ME")

In [20]:
totally = total[total.ZIP_distance_polo > 100]

In [21]:
totally = totally[totally.ZIP_distance_ppma > 100]

In [22]:
totally

,ME,ZIP_distance_polo,ZIP_distance_ppma,Unnamed: 0,ADDR_1_masterfile_POLO,ADDR_2_masterfile_POLO,ADDR_3_masterfile_POLO,CITY_masterfile_POLO,DATE_masterfile_POLO,ZIP_masterfile_POLO,...,ADDR_2_insurance_PPMA,ADDR_2_masterfile_PPMA,ADDR_3_insurance,ADDR_3_masterfile_PPMA,CITY_insurance,CITY_masterfile_PPMA,DATE_insurance,DATE_masterfile_PPMA,ZIP_insurance,ZIP_masterfile_PPMA
1,4201110101,106.769884,106.769884,1,4601 Ironbound Rd,Eastern State Hospital,NaN,Williamsburg,8/21/2019,23188,...,NaN,NaN,NaN,NaN,Arlington,Williamsburg,8/22/2019,8/21/2019,22206,23188.0
3,3401050352,848.279432,848.279432,3,1 University Of New Mexico,Hsc Department Of Pediatrics MSC10 5,NaN,Albuquerque,8/18/2019,87131,...,NaN,NaN,NaN,NaN,Sacramento,Albuquerque,8/20/2019,12/26/2016,95864,87131.0
14,2878161443,163.469229,161.205705,14,601 John St,Bronson Methodist Hospital - Emergency D,NaN,Kalamazoo,8/16/2019,49007,...,Carmel,Apt 104,NaN,NaN,Carmel,Kalamazoo,8/21/2019,8/1/2017,46032,49009.0
21,30501172827,186.089418,186.089418,21,1400 Medical Campus Dr,NaN,NaN,Traverse City,8/16/2019,49684,...,NaN,NaN,NaN,NaN,Auburn Hills,Traverse City,8/21/2019,8/16/2019,48326,49684.0
29,65401171317,110.295293,110.295293,29,1101 W University Dr 3,NaN,NaN,Rochester Hls,8/16/2019,48307,...,NaN,Apt 2,NaN,NaN,Eastlake,Rochester Hls,8/21/2019,1/16/2019,44095,48307.0
33,3401160719,372.485207,364.543608,33,2001 S Merriman Rd,Ste 100,NaN,Westland,8/16/2019,48186,...,NaN,Apt 603,NaN,NaN,Hummelstown,Dearborn,8/17/2019,8/16/2019,17036,48126.0
36,78105970055,559.409839,559.409839,36,777 37th St,Ste C107,Vero Renal Associates,Vero Beach,8/16/2019,32960,...,Ste 2101,Ste C107,NaN,Vero Renal Associates,Hickory,Vero Beach,8/17/2019,8/16/2019,28602,32960.0
37,3845160529,187.166765,187.166765,37,1250 S Washington St,NaN,NaN,Van Wert,8/16/2019,45891,...,NaN,NaN,NaN,NaN,Jeffersonvlle,Van Wert,8/17/2019,8/16/2019,47130,45891.0
42,49562810187,306.480607,306.480607,42,2103 Wythe Dr,NaN,NaN,Evans,8/13/2019,30809,...,NaN,NaN,NaN,NaN,Ocala,Evans,8/14/2019,8/13/2019,34480,30809.0
43,3601151315,476.176304,476.176304,43,3450 Hull Rd,Uf Health Center For Movement Disorder And Res...,NaN,Gainesville,8/13/2019,32607,...,NaN,Uf Health Center For Movement Disorder And Res...,NaN,NaN,Chapel Hill,Gainesville,8/14/2019,8/13/2019,27517,32607.0


In [23]:
totally.to_csv('far_places.csv')

In [ ]:
total.to_csv('zipcode_distances.csv')

In [10]:
def bucket_zip_distances(df, column_name):
    bins=[0,50,100,150,200,250,300,350,400,450,500,550,600,10000]
    labels = ["<50", "50-100", "100-150", "150-200","200-250","250-300","300-350","350-400","400-450","450-500", "500-550","550-600", ">600"]
    new_column_name = column_name + "_bins"
    df[new_column_name]=pd.cut(df[column_name], bins=bins, labels=labels)
    buckets = df[[new_column_name, column_name]].groupby(new_column_name).count()
#     buckets = buckets.apply(lambda x: x/df.shape[0]*100)
    return(buckets)

In [9]:
zipcode_distances = pd.read_csv("zipcode_distances.csv")

,ZIP_distance_polo
ZIP_distance_polo_bins,
<50,38.814547
50-100,5.522013
100-150,3.497996
150-200,2.777052
200-250,2.623311
250-300,2.075513
300-350,2.168802
350-400,1.764298
400-450,1.833705


In [53]:
bucket_zip_distances(updated_distances, 'ZIP_distance_polo_new')

,ZIP_distance_polo_new
ZIP_distance_polo_new_bins,
<50,2312
50-100,1634
100-150,1702
150-200,1824
200-250,2189
250-300,2024
300-350,2245
350-400,1879
400-450,2142


In [54]:
bucket_zip_distances(updated_distances, 'ZIP_distance_ppma')

,ZIP_distance_ppma
ZIP_distance_ppma_bins,
<50,2388
50-100,1606
100-150,1702
150-200,1784
200-250,2163
250-300,2062
300-350,2228
350-400,1840
400-450,2131


In [58]:
updated_distances[['ZIP_distance_ppma']].mean()

ZIP_distance_ppma    864.463724
dtype: float64

In [17]:
far = zipcode_distances[zipcode_distances.ZIP_distance_polo > 200]
far = far[far.ZIP_distance_ppma > 200]
far.shape[0]

42938

In [19]:
far.shape[0]/zipcode_distances.shape[0]

0.3204543588748498

In [59]:
military_count = 0
polo_apo = 0
polo_fpo = 0
polo_dpo = 0
ppma_apo = 0
ppma_fpo = 0
ppma_dpo = 0
insurance_apo = 0
insurance_fpo = 0
insurance_dpo = 0
dict_list=[]
for row in updated_distances.itertuples():
    new_dict ={}
    military = False
    new_dict["ME"]=row.ME
    if 'APO' in str(row.CITY_masterfile_POLO):
        polo_apo += 1
        military = True
    if 'FPO' in str(row.CITY_masterfile_POLO):
        polo_fpo += 1
        military = True
    if 'DPO' in str(row.CITY_masterfile_POLO):
        polo_dpo += 1
        military = True
    if 'APO' in str(row.CITY_insurance):
        insurance_apo += 1
        military = True
    if 'FPO' in str(row.CITY_insurance):
        insurance_fpo += 1
        military = True
    if 'DPO' in str(row.CITY_masterfile_POLO):
        insurance_dpo += 1
        military = True
    if 'APO' in str(row.CITY_masterfile_PPMA):
        ppma_apo += 1
        military = True
    if 'FPO' in str(row.CITY_masterfile_PPMA): 
        ppma_fpo += 1
        military = True
    if 'DPO' in str(row.CITY_masterfile_POLO):
        ppma_dpo += 1
        military = True
    if military == True:
        military_count +=1
        new_dict["Military"]=True
    if military == False:
        new_dict["Military"]=False
    dict_list.append(new_dict)

In [61]:
military_count

287

In [62]:
military = pd.DataFrame(dict_list)
x = pd.merge(military, far, on ="ME")
x[x.Military != True]

,ME,Military,Unnamed: 0,ZIP_distance_polo,ZIP_distance_ppma,Unnamed: 0.1,ADDR_1_masterfile_POLO,ADDR_2_masterfile_POLO,ADDR_3_masterfile_POLO,CITY_masterfile_POLO,...,ADDR_3_insurance,ADDR_3_masterfile_PPMA,CITY_insurance,CITY_masterfile_PPMA,DATE_insurance,DATE_masterfile_PPMA,ZIP_insurance,ZIP_masterfile_PPMA,ZIP_distance_polo_bins,ZIP_distance_ppma_bins
0,3401050352,False,3,848.279432,848.279432,3,1 University Of New Mexico,Hsc Department Of Pediatrics MSC10 5,NaN,Albuquerque,...,NaN,NaN,Sacramento,Albuquerque,8/20/2019,12/26/2016,95864,87131.0,>600,>600
1,3401160719,False,33,372.485207,364.543608,33,2001 S Merriman Rd,Ste 100,NaN,Westland,...,NaN,NaN,Hummelstown,Dearborn,8/17/2019,8/16/2019,17036,48126.0,350-400,350-400
2,78105970055,False,36,559.409839,559.409839,36,777 37th St,Ste C107,Vero Renal Associates,Vero Beach,...,NaN,Vero Renal Associates,Hickory,Vero Beach,8/17/2019,8/16/2019,28602,32960.0,550-600,550-600
3,49562810187,False,42,306.480607,306.480607,42,2103 Wythe Dr,NaN,NaN,Evans,...,NaN,NaN,Ocala,Evans,8/14/2019,8/13/2019,34480,30809.0,300-350,300-350
4,3601151315,False,43,476.176304,476.176304,43,3450 Hull Rd,Uf Health Center For Movement Disorder And Res...,NaN,Gainesville,...,NaN,NaN,Chapel Hill,Gainesville,8/14/2019,8/13/2019,27517,32607.0,450-500,450-500
5,5501090958,False,44,772.947116,772.947116,44,5323 Harry Hines Blvd # MC-9063,Ut Southwestern Medical Center,NaN,Dallas,...,NaN,NaN,Carmel,Dallas,8/14/2019,8/13/2019,46032,75390.0,>600,>600
6,4178130477,False,45,1967.690080,1967.690080,45,1100 N State St,Usc Lac,NaN,Los Angeles,...,NaN,NaN,Trenton,Los Angeles,8/20/2019,8/13/2019,48183,90033.0,>600,>600
7,519160897,False,47,2041.854144,2041.854144,47,1500 E Medical Center Dr,F6790 Uh South 5243,Educational Limited,Ann Arbor,...,NaN,Educational Limited,Sunnyvale,Ann Arbor,8/21/2019,8/13/2019,94086,48109.0,>600,>600
8,4815050768,False,49,242.832152,242.832152,49,1136 E Grande Blvd,Duncan F Hanby M D,NaN,Tyler,...,NaN,NaN,Lafayette,Tyler,8/14/2019,8/13/2019,70508,75703.0,200-250,200-250
9,5104100165,False,51,474.361606,474.361606,51,1600 SW Archer Rd,Pediatrics Division Of Pediatric Infecti,University Of Florida,Gainesville,...,NaN,University Of Florida,Durham,Gainesville,8/14/2019,8/13/2019,27713,32610.0,450-500,450-500
